In [1]:
# DEPENDENCIES
import copy #Allows us to create copies of objects in memory
import math #Math functionality
import numpy as np #Numpy for working with arrays
import matplotlib.pyplot as plt #Plotting functionality 
import matplotlib.colors #For colormap functionality
from mpl_toolkits.mplot3d import Axes3D #3D Plotting functionality
import ipywidgets as widgets
from numpy import genfromtxt #For importing structure data from csv
import csv #To export deflected coords to csv file

# 3D Space Frame Analysis - Transmission Tower
<mark>Notebook accompanying *'Case Study Structures'* section, lecture 33</mark>

## Structure Data Entry
<mark>**⚠️ Remember to locate your Vertices and Edges csv files in the same directory/folder as this Jupyter Notebook.** A Vertices and Edges csv file was packaged with this Notebook for convenience. You will need to replace these with your own versions after generating them in Blender.</mark>

In [2]:
#=================================START OF DATA ENTRY================================
#Constants
E = 200*10**9 #(N/m^2)
A = 0.005 #(m^2)
xFac = 50 #Scale factor for plotted displacements

#Import structure data
nodes = genfromtxt('Vertices.csv', delimiter=',')
members = genfromtxt('Edges.csv', delimiter=',')
members = np.int_(members) #Convert members definitions from float to int

#Supports [index starting at 1]
restrainedDoF = [1,2,3,10,11,12,151,152,153,154,155,156] #The degrees of freedom restrained by supports

#Loading [index starting at 0]
forceVector = np.array([np.zeros(len(nodes)*3)]).T

#Bottom Right
forceVector[176] = -25000
forceVector[179] = -25000

#Mid Right
forceVector[146] = -25000
forceVector[191] = -25000

#Top Right
forceVector[212] = -25000
forceVector[215] = -25000

# #Bottom Left
# forceVector[200] = -25000
# forceVector[203] = -25000

# #Mid Left
# forceVector[236] = -25000
# forceVector[239] = -25000

# #Top Left
# forceVector[248] = -25000
# forceVector[251] = -25000
# =================================END OF DATA ENTRY================================

## Plot structure to confirm before proceeding

In [3]:
def plotStructure(label_offset=0.01, xMargin=0.25, yMargin=0.25, zMargin=0.5, elevation=20, rotation=35): 
    
    fig = plt.figure() 
    axes = fig.add_axes([0.1,0.1,3,3],projection='3d') #Indicate a 3D plot 
    axes.view_init(elevation, rotation) #Set the viewing angle of the 3D plot

    #Set offset distance for node label
    dx = label_offset #x offset for node label
    dy = label_offset #y offset for node label
    dz = label_offset #z offset for node label

    #Provide space/margin around structure
    x_margin = xMargin #x-axis margin
    y_margin = yMargin #y-axis margin
    z_margin = zMargin #z-axis margin

    #Plot members
    for mbr in members:  
        node_i = mbr[0] #Node number for node i of this member
        node_j = mbr[1] #Node number for node j of this member   

        ix = nodes[node_i-1,0] #x-coord of node i of this member
        iy = nodes[node_i-1,1] #y-coord of node i of this member
        iz = nodes[node_i-1,2] #z-coord of node i of this member
        jx = nodes[node_j-1,0] #x-coord of node j of this member
        jy = nodes[node_j-1,1] #y-coord of node j of this member
        jz = nodes[node_j-1,2] #z-coord of node j of this member

        axes.plot3D([ix,jx],[iy,jy],[iz,jz],'b') #Plot 3D member

    #Plot nodes
    for n, node in enumerate(nodes):
        axes.plot3D([node[0]],[node[1]],[node[2]],'bo',ms=6) #Plot 3D node
        label = str(n+1) #The node number label string
        axes.text(node[0]+dx, node[1]+dy, node[2]+dz, label, fontsize=16) #Add node label

    #Set axis limits to provide margin around structure
    maxX = nodes.max(0)[0]
    maxY = nodes.max(0)[1]
    maxZ = nodes.max(0)[2]
    minX = nodes.min(0)[0]
    minY = nodes.min(0)[1]
    minZ = nodes.min(0)[2]
    axes.set_xlim([minX-x_margin,maxX+x_margin])
    axes.set_ylim([minY-y_margin,maxY+y_margin])
    axes.set_zlim([0,maxZ+z_margin])

    axes.set_xlabel('X-coordinate (m)')
    axes.set_ylabel('Y-coordinate (m)')
    axes.set_zlabel('Z-coordinate (m)')
    axes.set_title('Structure to analyse')
    axes.grid()

#Call the interact widget    
widgets.interact(plotStructure, 
                 label_offset=(0.01, 0.1, 0.01), 
                 xMargin=(0.25, 6, 0.25),
                 yMargin=(0.25, 6, 0.25),
                 zMargin=(0.5, 3, 0.25),
                 elevation=(0,360,10),
                 rotation=(0,360,10))
plt.show()

interactive(children=(FloatSlider(value=0.01, description='label_offset', max=0.1, min=0.01, step=0.01), FloatSlider(value=0.25, description='xMargin', max=6.0, min=0.25, step=0.25), FloatSlider(value=0.25, description='yMargin', max=6.0, min=0.25, step=0.25), FloatSlider(value=0.5, description='zMargin', max=3.0, min=0.5, step=0.25), IntSlider(value=20, description='elevation', max=360, step=10), IntSlider(value=35, description='rotation', max=360, step=10), Output()), _dom_classes=('widget-interact',))

## Calculate member orientation and length

In [4]:
#Define a function to calculate member orientation and length
def memberOrientation3D(memberNo):#Function title
    memberIndex = memberNo-1 #Index identifying member in array of members
    node_i = members[memberIndex][0] #Node number for node i of this member
    node_j = members[memberIndex][1] #Node number for node j of this member

    ix = nodes[node_i-1][0] #x-coord for node i (RENAME)
    iy = nodes[node_i-1][1] #y-coord for node i (RENAME)
    iz = nodes[node_i-1][2] #z-coord of node i 
    jx = nodes[node_j-1][0] #x-coord for node j (RENAME)
    jy = nodes[node_j-1][1] #y-coord for node j (RENAME)
    jz = nodes[node_j-1][2] #z-coord of node j
    
    #Angle of member with respect to horizontal axis    
    dx = jx-ix #x-component of vector along member
    dy = jy-iy #y-component of vector along member
    dz = jz-iz #z-component of vector along member
    mag = math.sqrt(dx**2 + dy**2 + dz**2) #Magnitude of vector (length of member)
 
    cos_theta_x = (jx-ix)/mag #Cosine of member angle w.r.t. x-axis
    cos_theta_y = (jy-iy)/mag #Cosine of member angle w.r.t. y-axis 
    cos_theta_z = (jz-iz)/mag #Cosine of member angle w.r.t. z-axis    
    
    return [cos_theta_x, cos_theta_y, cos_theta_z, mag]

In [5]:
#Calculate cosines (x3) and length for each member and store
cosX = np.array([])#Initialise an array to hold cos(theta_x)
cosY = np.array([])#Initialise an array to hold cos(theta_y)
cosZ = np.array([])#Initialise an array to hold cos(theta_z)
lengths = np.array([]) #Initialise an array to hold lengths
for n, mbr in enumerate(members):
    [ctx, cty, ctz, length] = memberOrientation3D(n+1)
    cosX = np.append(cosX,ctx)
    cosY = np.append(cosY,cty)
    cosZ = np.append(cosZ,ctz)
    lengths = np.append(lengths,length)

## Define a function to calculate member global stiffness matrix

In [6]:
#Define a function to calculate the global stiffness matrix of an axially loaded bar
def calculateKg3D(memberNo): 
    """
    Calculate the global stiffness matrix for an axially loaded bar
    memberNo: The member number 
    """        
    #Extract relevant member cosines and length
    x = cosX[memberNo-1]
    y = cosY[memberNo-1]
    z = cosZ[memberNo-1]
    mag = lengths[memberNo-1]

    #For clarity, define individual elements of global stiffness matrix
    #Row 1
    k11 = x**2
    k12 = x*y
    k13 = x*z
    k14 = -x**2
    k15 = -x*y
    k16 = -x*z  
    #Row 2
    k21 = x*y
    k22 = y**2
    k23 = y*z
    k24 = -x*y
    k25 = -y**2
    k26 = -y*z
    #Row 3
    k31 = x*z
    k32 = y*z
    k33 = z**2
    k34 = -x*z
    k35 = -y*z
    k36 = -z**2
    #Row 4
    k41 = -x**2
    k42 = -x*y
    k43 = -x*z
    k44 = x**2
    k45 = x*y
    k46 = x*z
    #Row 5
    k51 = -x*y
    k52 = -y**2
    k53 = -y*z
    k54 = x*y
    k55 = y**2
    k56 = y*z
    #Row 6
    k61 = -x*z
    k62 = -y*z
    k63 = -z**2
    k64 = x*z
    k65 = y*z
    k66 = z**2

    #Build the quadrants of the global stiffness matrix
    K11 = (E*A/mag)*np.array([[k11,k12,k13],[k21,k22,k23],[k31,k32,k33]]) #Top left quadrant of local stiffness matrix
    K12 = (E*A/mag)*np.array([[k14,k15,k16],[k24,k25,k26],[k34,k35,k36]]) #Top right quadrant of local stiffness matrix   
    K21 = (E*A/mag)*np.array([[k41,k42,k43],[k51,k52,k53],[k61,k62,k63]]) #Bottom left quadrant of local stiffness matrix   
    K22 = (E*A/mag)*np.array([[k44,k45,k46],[k54,k55,k56],[k64,k65,k66]]) #Bottom right quadrant of local stiffness matrix           
    
    return [K11, K12, K21,K22]

## Build the primary stiffness matrix, Kp

In [7]:
nDoF = np.amax(members)*3 #Total number of degrees of freedom in the problem
Kp = np.zeros([nDoF,nDoF]) #Initialise the primary stiffness matrix

for n, mbr in enumerate(members):
#note that enumerate adds a counter to an iterable (n)

    #Calculate the quadrants of the global stiffness matrix for the member
    [K11, K12, K21,K22] = calculateKg3D(n+1)

    node_i = mbr[0] #Node number for node i of this member
    node_j = mbr[1] #Node number for node j of this member
    
    #Primary stiffness matrix indices associated with each node
    #i.e. node 1 occupies indices 0, 1 and 2 (accessed in Python with [0:3])
    ia = 3*node_i-3 #index 0 (e.g. node 1)
    ib = 3*node_i-1 #index 2 (e.g. node 1)
    ja = 3*node_j-3 #index 3 (e.g. node 2)
    jb = 3*node_j-1 #index 5 (e.g. node 2)
    
    Kp[ia:ib+1,ia:ib+1] = Kp[ia:ib+1,ia:ib+1] + K11
    Kp[ia:ib+1,ja:jb+1] = Kp[ia:ib+1,ja:jb+1] + K12
    Kp[ja:jb+1,ia:ib+1] = Kp[ja:jb+1,ia:ib+1] + K21
    Kp[ja:jb+1,ja:jb+1] = Kp[ja:jb+1,ja:jb+1] + K22    

## Extract structure stiffness matrix, Ks

In [8]:
restrainedIndex = [x - 1 for x in restrainedDoF] #Index for each restrained DoF (list comprehension)

#Reduce to structure stiffness matrix by deleting rows and columns for restrained DoF
Ks = np.delete(Kp,restrainedIndex,0) #Delete rows
Ks = np.delete(Ks,restrainedIndex,1) #Delete columns
Ks = np.matrix(Ks) # Convert Ks from numpy.ndarray to numpy.matrix to use build in inverter function

## Solve for displacements

In [9]:
forceVectorRed = copy.copy(forceVector)# Make a copy of forceVector so the copy can be edited, leaving the original unchanged
forceVectorRed = np.delete(forceVectorRed,restrainedIndex,0) #Delete rows corresponding to restrained DoF
U = Ks.I*forceVectorRed

## Solve for reactions

In [10]:
#Construct the global displacement vector
UG = np.zeros(nDoF) #Initialise an array to hold the global displacement vector
c=0 #Initialise a counter to track how many restraints have been imposed
for i in np.arange(nDoF):    
    if i in restrainedIndex:
        #Impose zero displacement
        UG[i] = 0        
    else:
        #Assign actual displacement
        UG[i] = U[c]
        c=c+1

UG = np.array([UG]).T  
FG = np.matmul(Kp,UG)

#Generate output statements
for i in np.arange(0,len(restrainedIndex)):           
    index = restrainedIndex[i]

## Solve for member forces

In [11]:
mbrForces = np.array([]) #Initialise an array to hold member forces
for n, mbr in enumerate(members):    
    mag = lengths[n]
    x = cosX[n]
    y = cosY[n]
    z = cosZ[n]
    
    node_i = mbr[0] #Node number for node i of this member
    node_j = mbr[1] #Node number for node j of this member    
    #Primary stiffness matrix indices associated with each node
    ia = 3*node_i-3 #index 0 (e.g. node 1)
    ib = 3*node_i-1 #index 2 (e.g. node 1)
    ja = 3*node_j-3 #index 3 (e.g. node 2)
    jb = 3*node_j-1 #index 5 (e.g. node 2)   
    
    T = np.array([[x,y,z,0,0,0],[0,0,0,x,y,z]]) 
    
    disp = np.array([ [UG[ia,0],UG[ia+1,0],UG[ib,0],UG[ja,0],UG[ja+1,0],UG[jb,0]]]).T #Global displacements
    disp_local = np.matmul(T,disp) #Local displacements    
    F_axial = (A*E/mag)*(disp_local[1]-disp_local[0]) #Axial loads    
    mbrForces = np.append(mbrForces,F_axial) #Store axial loads

## Plot Axial Forces

In [12]:
def plotForces(xMargin=6, yMargin=6, zMargin=0.5, elevation=20, rotation=35): 

    fig = plt.figure() 
    axes = fig.add_axes([0.1,0.1,2.5,2.5],projection='3d') 
    axes.view_init(elevation, rotation)

    #Provide space/margin around structure
    x_margin = xMargin #x-axis margin
    y_margin = yMargin #y-axis margin
    z_margin = zMargin #z-axis margin

    #Create color scale for member forces
    cmap = plt.cm.seismic_r #Define the color scale to use (note _r reverses colourmap)

    #norm = matplotlib.colors.Normalize(vmin=mbrForces.min(0), vmax=mbrForces.max(0)) #Normalise colour scale to limits of my data
    norm = matplotlib.colors.DivergingNorm(vmin=mbrForces.min(0), 
                                           vcenter=0, 
                                           vmax=mbrForces.max(0))#set midpoint of colormap to zero
    
    #Add color scale to figure
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    fig.colorbar(sm) 

    #Plot members
    for n, mbr in enumerate(members):  
        node_i = mbr[0] #Node number for node i of this member
        node_j = mbr[1] #Node number for node j of this member   

        ix = nodes[node_i-1,0] #x-coord of node i of this member
        iy = nodes[node_i-1,1] #y-coord of node i of this member
        iz = nodes[node_i-1,2] #z-coord of node i of this member
        jx = nodes[node_j-1,0] #x-coord of node j of this member
        jy = nodes[node_j-1,1] #y-coord of node j of this member
        jz = nodes[node_j-1,2] #z-coord of node j of this member

        #Index of DoF for this member
        ia = 3*node_i-3 #index 0 (e.g. node 1)
        ib = 3*node_i-1 #index 2 (e.g. node 1)
        ja = 3*node_j-3 #index 3 (e.g. node 2)
        jb = 3*node_j-1 #index 5 (e.g. node 2)

        if(abs(mbrForces[n])>0.001):         
            axes.plot3D([ix,jx],[iy,jy],[iz,jz],color=cmap(norm(mbrForces[n])))#Non-zero force in member
        else:
            axes.plot3D([ix,jx],[iy,jy],[iz,jz],'grey',linestyle='--') #Zero-force member


    #Plot nodes
    for node in nodes:
        axes.plot3D([node[0]],[node[1]],[node[2]],'bo', ms=3) 

    #Set axis limits to provide margin around structure
    maxX = nodes.max(0)[0]
    maxY = nodes.max(0)[1]
    maxZ = nodes.max(0)[2]
    minX = nodes.min(0)[0]
    minY = nodes.min(0)[1]
    minZ = nodes.min(0)[2]

    axes.set_xlim([minX-x_margin,maxX+x_margin])
    axes.set_ylim([minY-y_margin,maxY+y_margin])
    axes.set_zlim([0,maxZ+z_margin])    
    axes.set_xlabel('X-coordinate (m)')
    axes.set_ylabel('Y-coordinate (m)')
    axes.set_zlabel('Z-coordinate (m)')
    axes.set_title('Tension/compression members')
    axes.grid()
    
#Call the interact widget    
widgets.interact(plotForces, 
                 xMargin=(0.25, 6, 0.25),
                 yMargin=(0.25, 6, 0.25),
                 zMargin=(0.5, 3, 0.25),
                 elevation=(0,360,10),
                 rotation=(0,360,10))
plt.show()

interactive(children=(FloatSlider(value=6.0, description='xMargin', max=6.0, min=0.25, step=0.25), FloatSlider(value=6.0, description='yMargin', max=6.0, min=0.25, step=0.25), FloatSlider(value=0.5, description='zMargin', max=3.0, min=0.5, step=0.25), IntSlider(value=20, description='elevation', max=360, step=10), IntSlider(value=35, description='rotation', max=360, step=10), Output()), _dom_classes=('widget-interact',))

## Plot Deflected Shape

In [13]:
def plotDeflection(xMargin=6, yMargin=6, zMargin=0.5, elevation=20, rotation=35, scaleFactor=1): 

    fig = plt.figure() 
    axes = fig.add_axes([0.1,0.1,2.5,2.5],projection='3d') 
    axes.view_init(elevation, rotation)
    
    #Provide space/margin around structure
    x_margin = xMargin #x-axis margin
    y_margin = yMargin #y-axis margin
    z_margin = zMargin #z-axis margin

    #Plot members
    for mbr in members:  
        node_i = mbr[0] #Node number for node i of this member
        node_j = mbr[1] #Node number for node j of this member   

        ix = nodes[node_i-1,0] #x-coord of node i of this member
        iy = nodes[node_i-1,1] #y-coord of node i of this member
        iz = nodes[node_i-1,2] #z-coord of node i of this member
        jx = nodes[node_j-1,0] #x-coord of node j of this member
        jy = nodes[node_j-1,1] #y-coord of node j of this member
        jz = nodes[node_j-1,2] #z-coord of node j of this member

        #Index of DoF for this member   
        ia = 3*node_i-3 #index 0 (e.g. node 1)
        ib = 3*node_i-1 #index 2 (e.g. node 1)
        ja = 3*node_j-3 #index 3 (e.g. node 2)
        jb = 3*node_j-1 #index 5 (e.g. node 2)

        axes.plot3D([ix,jx],[iy,jy],[iz,jz],'grey', lw=0.75) #Member
        axes.plot3D([ix + UG[ia,0]*scaleFactor, jx + UG[ja,0]*scaleFactor], 
                    [iy + UG[ia+1,0]*scaleFactor, jy + UG[ja+1,0]*scaleFactor],
                    [iz + UG[ib,0]*scaleFactor, jz + UG[jb,0]*scaleFactor],
                  'r') #Deformed member 

    maxX = nodes.max(0)[0]
    maxY = nodes.max(0)[1]
    maxZ = nodes.max(0)[2]
    minX = nodes.min(0)[0]
    minY = nodes.min(0)[1]
    minZ = nodes.min(0)[2]

    axes.set_xlim([minX-x_margin,maxX+x_margin])
    axes.set_ylim([minY-y_margin,maxY+y_margin])
    axes.set_zlim([0,maxZ+z_margin])
    axes.set_xlabel('X-coordinate (m)')
    axes.set_ylabel('Y-coordinate (m)')
    axes.set_zlabel('Z-coordinate (m)')
    axes.set_title('Deflected shape')
    axes.grid()
    
#Call the interact widget    
widgets.interact(plotDeflection, 
                 xMargin=(0.25, 6, 0.25),
                 yMargin=(0.25, 6, 0.25),
                 zMargin=(0.5, 3, 0.25),
                 elevation=(0,360,10),
                 rotation=(0,360,10),
                 scaleFactor=(1,500,50))
plt.show()

interactive(children=(FloatSlider(value=6.0, description='xMargin', max=6.0, min=0.25, step=0.25), FloatSlider(value=6.0, description='yMargin', max=6.0, min=0.25, step=0.25), FloatSlider(value=0.5, description='zMargin', max=3.0, min=0.5, step=0.25), IntSlider(value=20, description='elevation', max=360, step=10), IntSlider(value=35, description='rotation', max=360, step=10), IntSlider(value=1, description='scaleFactor', max=500, min=1, step=50), Output()), _dom_classes=('widget-interact',))

## Summary output

In [14]:
#Generate output statements
print("REACTIONS")
for i in np.arange(0,len(restrainedIndex)):           
    index = restrainedIndex[i]
    print("Reaction at DoF {one}: {two} kN".format(one = index+1, two = round(FG[index].item()/1000,2)))

print("")   
print("MEMBER FORCES")    
for n, mbr in enumerate(members):    
    print("Force in member {one} (nodes {two} to {three}) is {four} kN".format(one = n+1, two=mbr[0], three=mbr[1], four=round(mbrForces[n]/1000,2)))

print("")   
print("NODAL DISPLACEMENTS") 
for n, node in enumerate(nodes):    

    ix = 3*(n+1)-3 #x DoF for this node
    iy = 3*(n+1)-2 #y DoF for this node
    iz = 3*(n+1)-1 #y DoF for this node
    
    ux = round(UG[ix,0],5) #x nodal displacement
    uy = round(UG[iy,0],5) #y nodal displacement
    uz = round(UG[iz,0],5) #z nodal displacement
    print("Node {one}: Ux = {two} m, Uy = {three} m, Uz = {four} m".format(one=n+1, two=ux, three=uy, four = uz))

REACTIONS
Reaction at DoF 1: 10.69 kN
Reaction at DoF 2: -7.15 kN
Reaction at DoF 3: -9.87 kN
Reaction at DoF 10: -10.69 kN
Reaction at DoF 11: 28.53 kN
Reaction at DoF 12: 84.87 kN
Reaction at DoF 151: -10.69 kN
Reaction at DoF 152: -28.53 kN
Reaction at DoF 153: 84.87 kN
Reaction at DoF 154: 10.69 kN
Reaction at DoF 155: 7.15 kN
Reaction at DoF 156: -9.87 kN

MEMBER FORCES
Force in member 1 (nodes 8 to 4) is -60.1 kN
Force in member 2 (nodes 47 to 55) is -0.0 kN
Force in member 3 (nodes 11 to 8) is -60.1 kN
Force in member 4 (nodes 15 to 11) is -109.2 kN
Force in member 5 (nodes 11 to 9) is 10.39 kN
Force in member 6 (nodes 9 to 10) is -11.13 kN
Force in member 7 (nodes 9 to 12) is 38.72 kN
Force in member 8 (nodes 16 to 15) is -123.88 kN
Force in member 9 (nodes 15 to 12) is 3.87 kN
Force in member 10 (nodes 14 to 15) is 12.22 kN
Force in member 11 (nodes 20 to 16) is -131.6 kN
Force in member 12 (nodes 22 to 19) is -142.05 kN
Force in member 13 (nodes 23 to 20) is -142.05 kN
Force 

## Export new nodal positions to csv
<mark>⚠️ **Remember to update your file path**</mark>

In [15]:
#Export Deflected shape
exportDeflectionFactor = 100
newCoords = np.empty((0,3),int)
for n, node in enumerate(nodes):   
    x = nodes[n,0] #x-coord of node n
    y = nodes[n,1] #y-coord of node n
    z = nodes[n,2] #z-coord of node n
    
    ix = 3*(n+1)-3 #x DoF for this node
    iy = 3*(n+1)-2 #y DoF for this node
    iz = 3*(n+1)-1 #y DoF for this node
    
    ux = UG[ix,0]*exportDeflectionFactor #x nodal displacement
    uy = UG[iy,0]*exportDeflectionFactor #y nodal displacement
    uz = UG[iz,0]*exportDeflectionFactor #y nodal displacement
    
    newX = x+ux #Deflected x-coord
    newY = y+uy #Deflected y-coord
    newZ = z+uz #Deflected z-coord   
    
    nodePos = np.array([newX, newY, newZ])
    newCoords = np.append(newCoords, [nodePos], axis=0)
        
#Export vertex coordinates to CSV file 
filename = "/Path/to/file/Deflected-Vertices.csv"
# writing to csv file  
with open(filename, 'w') as csvfile:  
    csvwriter = csv.writer(csvfile)  # creating a csv writer object  
    csvwriter.writerows(newCoords) # writing the data rows   